In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('kmeans').getOrCreate()

In [3]:
dataset = sqlContext.sql("SELECT * FROM seeds_dataset_csv")

In [4]:
dataset.printSchema()

root
-- area: double (nullable = true)
-- perimeter: double (nullable = true)
-- compactness: double (nullable = true)
-- length_of_kernel: double (nullable = true)
-- width_of_kernel: double (nullable = true)
-- asymmetry_coefficient: double (nullable = true)
-- length_of_groove: double (nullable = true)

In [5]:
dataset.head(1)

Out[5]: [Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)]

In [6]:
from pyspark.ml.clustering import KMeans

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
dataset.columns

Out[8]: ['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [9]:
assembler = VectorAssembler(inputCols=dataset.columns, outputCol='features')

In [10]:
final_data = assembler.transform(dataset)

In [11]:
final_data.printSchema()

root
-- area: double (nullable = true)
-- perimeter: double (nullable = true)
-- compactness: double (nullable = true)
-- length_of_kernel: double (nullable = true)
-- width_of_kernel: double (nullable = true)
-- asymmetry_coefficient: double (nullable = true)
-- length_of_groove: double (nullable = true)
-- features: vector (nullable = true)

In [12]:
from pyspark.ml.feature import StandardScaler

In [13]:
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')

In [14]:
scaler_model = scaler.fit(final_data)

In [15]:
final_data = scaler_model.transform(final_data)

In [16]:
final_data.head(1)

Out[16]: [Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

In [17]:
kmeans = KMeans(featuresCol='scaledFeatures', k=3)

In [18]:
model = kmeans.fit(final_data)

In [19]:
print("WSSSE")
print(model.computeCost(final_data))

WSSSE
429.07559671506715

In [20]:
print("Centers")
print(model.clusterCenters())

Centers
[array([ 4.87257659, 10.88120146, 37.27692543, 12.3410157 , 8.55443412,
 1.81649011, 10.32998598]), array([ 4.06105916, 10.13979506, 35.80536984, 11.82133095, 7.50395937,
 3.27184732, 10.42126018]), array([ 6.31670546, 12.37109759, 37.39491396, 13.91155062, 9.748067 ,
 2.39849968, 12.2661748 ])]

In [21]:
model.transform(final_data).select('prediction').show()

+----------+
prediction|
+----------+
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 2|
 2|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 1|
+----------+
only showing top 20 rows

In [22]:
model.transform(final_data).groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 65|
 2| 70|
 0| 75|
+----------+-----+